In [88]:
import folium
import folium.plugins as plugins
import pickle
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import datetime
import ipywidgets as widgets

In [89]:
class trail:
    def __init__(self,nom_id,d,s,hd,csv):
        self.nom_id = nom_id
        self.date = d
        self.suivi = s
        self.heure_depart =hd
        self.csv_file =csv
        
    def pred_temps(self, vit_plat,vit_Dplus,ralentissement,durée_stop,à_partir_de):
        print('Opti.: Vit_plat_km/h:',vit_plat,' | Vit_den+_m/h:', vit_Dplus,' | Ralent.:',ralentissement)
        df = self.df_ppassage
        df['Pred_Cum_T_Rom']=0 
        df['Pred_T_Rom']=0
        df['Pred_T_Rom']=round(df['Delta_Dist_km']/vit_plat+df['Delta_D+_m']/vit_Dplus+df['Cumul_Dist_km']*ralentissement,2)
        # df['Heure_predite']= 
        for i in range(1,len(df)):             
            df.loc[i,'Pred_Cum_T_Rom']=df.loc[i-1,'Pred_Cum_T_Rom']+df.loc[i,'Pred_T_Rom']
        for i in range(1,len(df)): 
            if df.loc[i,'Pred_Cum_T_Rom']>à_partir_de:
                df.loc[i,'Pred_Cum_T_Rom']=df.loc[i,'Pred_Cum_T_Rom']+durée_stop
            df.loc[i,'Heure_predite']=(pd.to_datetime(f'{self.date} {self.heure_depart}')+pd.Timedelta(hours=(df.loc[i,'Pred_Cum_T_Rom']//1), minutes= (df.loc[i,'Pred_Cum_T_Rom']%1*60))).strftime("%a. %H:%M")         
            if df.loc[0,'H_reel_h_init']!='Not runned':
                df.loc[i,'DELTA']=round(df.loc[i,'Pred_Cum_T_Rom']-df.loc[i,'H_reel_h_init'],2)
                df['H_reel_h']=round(df['H_reel_h'],2)
            else:
                df['DELTA']=0
        return df
    
    def plot_parcours(self,vit_plat,vit_Dplus,ralentissement):
        pts = [ (x,y) for x,y in zip(self.df_parcours['lat'],self.df_parcours['lon'])]
        map = folium.Map(location=[self.df_parcours['lat'].mean(), self.df_parcours['lon'].mean()], zoom_start=12)
        folium.PolyLine(pts).add_to(map)
        geo_df_list = [ (x,y) for x,y in zip(self.df_ppassage['lat'],self.df_ppassage['lon'])]
        for i, coordinates in enumerate(geo_df_list):
            txt = ''
            ic ='cloud'   
            couleur='blue'
            popContent = f''' {str(self.df_ppassage.loc[i,'Point_passage'])}
                            <style>
                            table, th, td {{
                              border: 1px solid black;
                              border-collapse: collapse;
                              text-align: center; 
                                }}
                            </style> 
                            <table style="width:100%">
                              <tr>
                                <th>Distance:</th>
                                <th>Altitude:</th>
                                <th>D+:</th>
                              </tr>
                              <tr>
                               <td>{str(self.df_ppassage.loc[i,'Cumul_Dist_km'])} km</td>
                               <td>{str(self.df_ppassage.loc[i,'Altitude_m'])} m</td>
                               <td>{str(self.df_ppassage.loc[i,'Cumul_D+_m'])} m</td>
                               </tr>
                               <tr>
                                <th>Heure Passage:</th>
                                <th>Heure Predite:</th>
                               </tr>
                               <tr>
                               <td>{str(self.df_ppassage.loc[i,'Heure_Passage'])}</td>
                               <td>{str(self.df_ppassage.loc[i,'Heure_predite'])}</td>
                               </tr>
                               </table> '''
            iframe = folium.IFrame(popContent)
            popup1 = folium.Popup(iframe,
                                  min_width=350,
                                  max_width=400)   
            folium.Marker(location=coordinates,
                           popup = popup1,            
                           icon=plugins.BeautifyIcon(
                           icon="arrow-down", icon_shape="marker",
                           number=i,
                           )).add_to(map)
        #     folium.Marker(location=coordinates,
        #                    popup = popup1,            
        #                    icon=folium.Icon(color=couleur,icon=ic,prefix='fa')).add_to(map)
        # map.save(f'{self.nom_id}_{vit_plat}_{vit_Dplus}_{ralentissement}.html')
        display(map)    

In [90]:
Lst_trails = pd.read_csv('ListeDesTrails.csv',sep=',')

In [91]:
PIK = "trails.dat"
Trails_objects = []
with open(PIK, "rb") as f:
    while True:
        try:
            Trails_objects.append(pickle.load(f))
        except EOFError:
            break

In [92]:
for i,T in enumerate(Trails_objects):
    print(i,T.nom_id,T.date,T.suivi,T.heure_depart,T.csv_file)

0 20195REFUGES 2019-07-28 livetrail 07:30:00 2019-07-28_5ref_31.7km.csv
1 2019MCC 2019-08-26 livetrail 10:00:00 2019-08-26_MCC_39.1km.csv
2 2020GTC 2020-09-13 livetrail 06:00:00 2020-09-13_GTC_56.5km.csv
3 20218REFUGES 2021-07-25 livetrail 05:00:00 2021-07-25_Tour des 8 refuges Evadict_65.1km.csv
4 2021DOOBLE 2021-07-31 livetrail 08:00:00 2021-07-31_LA DOOBLE - 16 KM_17.9km.csv
5 2021TRAVNORD 2021-08-21 livetrail 05:15:00 2021-08-21_Traversée Nord_83.4km.csv
6 2022UTHG 2022-06-17 livetrail  23:45:00 2022-06-17_Ultra-Trail® du haut-Giffre_97.4km.csv
7 2022MBM 2022-06-25 livetrail 08:00:00 2022-06-24_23km du Mont-Blanc_24.4km.csv
8 2022UTB 2022-07-16 livetrack 04:00:00 2022-07-16_Ultra Tour du Beaufortain 2022_114.0km.csv
9 2023UTLAC 2023-06-16 livetrail 19:00:00 2023-06-16_2023UTLAC_128.9km.csv
10 2023TDS 2023-08-28 livetrail 23:50:00 2023-08-28_2023TDS_147.6km.csv
11 All 1979-12-05 NA NA NA


# Visualisation des résultats et predictions

In [93]:
durée_arret = 1.25
à_partir_de = 13
def pred(i,vit_plat,vit_Dplus,ralentissement):
    df_p = Trails_objects[i].pred_temps(vit_plat,vit_Dplus,ralentissement,durée_arret,à_partir_de)
    display(df_p[['Point_passage','Cumul_Dist_km','Cumul_D+_m','Pred_Cum_T_Rom','Heure_Passage','Heure_predite','DELTA']])
    return df_p

In [94]:
Trails_objects[9].df_ppassage

,id,Point_passage,Delta_Dist_km,Cumul_Dist_km,Delta_D+_m,Cumul_D+_m,lon,lat,Altitude_m,Heure_Passage,H_reel_h,H_reel_h_init,Delta_reel_h,Delta_reel_h_init,Pred_Cum_T_Rom,Pred_T_Rom,Heure_predite
0,0,Départ Samoëns,0.0,0.0,0,0,6.72695,46.0835,708,Not runned,0,0,0.0,0.0,0.00,0.00,Fri. 19:00
1,4,Pointe d'Angolon,16.7,16.7,1797,1797,6.73103,46.1379,2060,Not runned,0,0,0.0,0.0,3.79,3.79,Fri. 22:47
2,6,Mines d'Or,7.2,23.9,166,1963,6.76556,46.1486,1388,Not runned,0,0,0.0,0.0,4.95,1.16,Fri. 23:57
3,8,Col de Coux,3.0,26.9,525,2488,6.79276,46.1502,1915,Not runned,0,0,0.0,0.0,5.79,0.84,Sat. 00:47
4,10,Col de la Golèse,5.5,32.4,241,2729,6.76099,46.1305,1661,Not runned,0,0,0.0,0.0,6.76,0.97,Sat. 01:45
5,12,Col de Bostan,4.9,37.3,706,3435,6.80974,46.1343,2273,Not runned,0,0,0.0,0.0,8.01,1.25,Sat. 03:00
6,14,Le Crêt,7.6,44.9,18,3453,6.74487,46.1073,1018,Not runned,0,0,0.0,0.0,9.11,1.10,Sat. 04:06
7,22,Boret,13.6,58.5,1460,4913,6.84522,46.1020,1110,Not runned,0,0,0.0,0.0,12.20,3.09,Sat. 07:12
8,24,Pelly,7.7,66.2,310,5223,6.83437,46.0746,938,Not runned,0,0,0.0,0.0,14.79,1.34,Sat. 09:47
9,26,Praz Commune,4.6,70.8,738,5961,6.82114,46.0577,1677,Not runned,0,0,0.0,0.0,16.02,1.23,Sat. 11:01


In [95]:
def parcours_plot(i,vit_plat,vit_Dplus,ralentissement):
    Trails_objects[i].plot_parcours(vit_plat,vit_Dplus,ralentissement)

In [97]:
Trail_selection = widgets.Dropdown(
    options=[(Trails_objects[i].nom_id,i) for i in range(len(Trails_objects))],
    value=9,
    description='Trail:',
)
vitesse_plat = widgets.FloatSlider(
    value=Trails_objects[-1].vit_plat,
    min=4.0,
    max=15.0,
    step=0.1,
    description='Vitesse plat:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
)
vitesse_Dplus = widgets.FloatSlider(
    value=Trails_objects[-1].vit_Dplus,
    min=600,
    max=1900,
    step=25,
    description='Vitesse D+:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
)
Ralentissement = widgets.FloatSlider(
    value=Trails_objects[-1].ralentissement,
    min=0,
    max=0.01,
    step=0.001,
    description='Ralentissement:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.5f',
)
ui = widgets.HBox([Trail_selection, vitesse_plat,vitesse_Dplus,Ralentissement])
res= widgets.interactive_output(pred,{'i':Trail_selection,'vit_plat':vitesse_plat,'vit_Dplus':vitesse_Dplus,'ralentissement':Ralentissement})
graph = widgets.interactive_output(parcours_plot,{'i':Trail_selection,'vit_plat':vitesse_plat,'vit_Dplus':vitesse_Dplus,'ralentissement':Ralentissement})
out = widgets.HBox([graph,res])

# display(ui,out)
display(graph,ui,res)

Output()

Output()

In [9]:
Lst_trails

,Trail,Date,Suivi,Heure_depart,Lien_URL,Fichier_parcours_gpx,Vit_plat_km/h,Vit_den+_m/h,Csv_file,Date_dt
0,20195REFUGES,2019-07-28,livetrail,07:30:00,https://livetrail.net/histo/fiz_2019/coureur.php,NaN,10.0,1229.8,2019-07-28_5ref_31.7km.csv,2019-07-28
1,2019MCC,2019-08-26,livetrail,10:00:00,https://livetrail.net/histo/utmb_2019/coureur.php,tracedetrail_94254.gpx,6.9,2933.5,2019-08-26_MCC_39.1km.csv,2019-08-26
2,2020GTC,2020-09-13,livetrail,06:00:00,https://livetrail.net/histo/serreche_2020/cour...,NaN,7.0,1314.0,2020-09-13_GTC_56.5km.csv,2020-09-13
3,20218REFUGES,2021-07-25,livetrail,05:00:00,https://fiz.livetrail.net/coureur.php,tracedetrail_156428.gpx,8.4,1217.6,2021-07-25_Tour des 8 refuges Evadict_65.1km.csv,2021-07-25
4,2021DOOBLE,2021-07-31,livetrail,08:00:00,https://megevenaturetrail.livetrail.run/coureu...,NaN,13.5,1643.0,2021-07-31_LA DOOBLE - 16 KM_17.9km.csv,2021-07-31
5,2021TRAVNORD,2021-08-21,livetrail,05:15:00,https://echappeebelle.livetrail.run/parcours....,tracedetrail_135447.gpx,5.2,2137.3,2021-08-21_Traversée Nord_83.4km.csv,2021-08-21
6,2022UTHG,2022-06-17,livetrail,23:45:00,https://ultratrailduhautgiffre.livetrail.run/c...,tracedetrail_166975.gpx,6.0,1283.3,2022-06-17_Ultra-Trail® du haut-Giffre_97.4km.csv,2022-06-17
7,2022MBM,2022-06-25,livetrail,08:00:00,https://mbm.livetrail.run/coureur.php,23km-2022.gpx,14.3,882.0,2022-06-24_23km du Mont-Blanc_24.4km.csv,2022-06-25
8,2022UTB,2022-07-16,livetrack,04:00:00,https://www.livetrack.me/fr/evenement/ultratou...,ultra_tour_du_beaufortain_2022-13673263-165493...,5.4,1177.4,2022-07-16_Ultra Tour du Beaufortain 2022_114....,2022-07-16
9,2023UTLAC,2023-06-16,livetrail,19:00:00,https://ultratrailduhautgiffre.livetrail.run/p...,tracedetrail_189127.gpx,NaN,NaN,NaN,2023-06-16


In [10]:
df2 = pred(8,6,1283,0)
df2['UTHG 6 1283 0']=df2['Heure_predite']
df2 = pred(8,5.2,2137,0)
df2['TRAV 5.2 2137 0']=df2['Heure_predite']
df2 = pred(8,5.8,1275,0)
df2['UTHG+TRAV 5.8 1275 0']=df2['Heure_predite']
df2 = pred(8,6.9,1284,0)
df2['ALL 6.9 1284 0']=df2['Heure_predite']
# df2 = pred(8,8.8,949,0.00559)
# df2['ALL 8.8 949 0.00559']=df2['Heure_predite']

Opti.: Vit_plat_km/h: 6  | Vit_den+_m/h: 1283  | Ralent.: 0


,Point_passage,Cumul_Dist_km,Cumul_D+_m,Pred_Cum_T_Rom,Heure_Passage,Heure_predite,DELTA
0,Départ,0.0,0,0.00,04:00:49,NaN,NaN
1,Gîte de Molliessoulaz,4.0,422,1.00,04:44:29,Sat. 05:00,0.27
2,Les Chappes,7.0,898,1.87,05:21:02,Sat. 05:52,0.53
3,Frêtte basse,9.0,1453,2.64,06:09:26,Sat. 06:38,0.50
4,Col des Lacs,13.0,1771,3.55,06:53:42,Sat. 07:33,0.67
5,Chalet du Soufflet,16.0,1777,4.05,07:16:23,Sat. 08:03,0.79
6,Ardoisière de Cevins,18.0,1991,4.55,07:56:40,Sat. 08:33,0.62
7,Col des Combettes,21.0,2254,5.25,08:37:22,Sat. 09:15,0.64
8,Col des Bonnets rouges,25.0,2452,6.07,09:22:21,Sat. 10:04,0.71
9,Lac de St Guerin,30.0,2465,6.91,10:02:31,Sat. 10:54,0.88


Opti.: Vit_plat_km/h: 5.2  | Vit_den+_m/h: 2137  | Ralent.: 0


,Point_passage,Cumul_Dist_km,Cumul_D+_m,Pred_Cum_T_Rom,Heure_Passage,Heure_predite,DELTA
0,Départ,0.0,0,0.00,04:00:49,NaN,NaN
1,Gîte de Molliessoulaz,4.0,422,0.97,04:44:29,Sat. 04:58,0.24
2,Les Chappes,7.0,898,1.77,05:21:02,Sat. 05:46,0.43
3,Frêtte basse,9.0,1453,2.41,06:09:26,Sat. 06:24,0.27
4,Col des Lacs,13.0,1771,3.33,06:53:42,Sat. 07:19,0.45
5,Chalet du Soufflet,16.0,1777,3.91,07:16:23,Sat. 07:54,0.65
6,Ardoisière de Cevins,18.0,1991,4.39,07:56:40,Sat. 08:23,0.46
7,Col des Combettes,21.0,2254,5.09,08:37:22,Sat. 09:05,0.48
8,Col des Bonnets rouges,25.0,2452,5.95,09:22:21,Sat. 09:57,0.59
9,Lac de St Guerin,30.0,2465,6.92,10:02:31,Sat. 10:55,0.89


Opti.: Vit_plat_km/h: 5.8  | Vit_den+_m/h: 1275  | Ralent.: 0


,Point_passage,Cumul_Dist_km,Cumul_D+_m,Pred_Cum_T_Rom,Heure_Passage,Heure_predite,DELTA
0,Départ,0.0,0,0.00,04:00:49,NaN,NaN
1,Gîte de Molliessoulaz,4.0,422,1.02,04:44:29,Sat. 05:01,0.29
2,Les Chappes,7.0,898,1.91,05:21:02,Sat. 05:54,0.57
3,Frêtte basse,9.0,1453,2.69,06:09:26,Sat. 06:41,0.55
4,Col des Lacs,13.0,1771,3.63,06:53:42,Sat. 07:37,0.75
5,Chalet du Soufflet,16.0,1777,4.15,07:16:23,Sat. 08:09,0.89
6,Ardoisière de Cevins,18.0,1991,4.66,07:56:40,Sat. 08:39,0.73
7,Col des Combettes,21.0,2254,5.38,08:37:22,Sat. 09:22,0.77
8,Col des Bonnets rouges,25.0,2452,6.22,09:22:21,Sat. 10:13,0.86
9,Lac de St Guerin,30.0,2465,7.09,10:02:31,Sat. 11:05,1.06


Opti.: Vit_plat_km/h: 6.9  | Vit_den+_m/h: 1284  | Ralent.: 0


,Point_passage,Cumul_Dist_km,Cumul_D+_m,Pred_Cum_T_Rom,Heure_Passage,Heure_predite,DELTA
0,Départ,0.0,0,0.00,04:00:49,NaN,NaN
1,Gîte de Molliessoulaz,4.0,422,0.91,04:44:29,Sat. 04:54,0.18
2,Les Chappes,7.0,898,1.72,05:21:02,Sat. 05:43,0.38
3,Frêtte basse,9.0,1453,2.44,06:09:26,Sat. 06:26,0.30
4,Col des Lacs,13.0,1771,3.27,06:53:42,Sat. 07:16,0.39
5,Chalet du Soufflet,16.0,1777,3.71,07:16:23,Sat. 07:42,0.45
6,Ardoisière de Cevins,18.0,1991,4.17,07:56:40,Sat. 08:10,0.24
7,Col des Combettes,21.0,2254,4.81,08:37:22,Sat. 08:48,0.20
8,Col des Bonnets rouges,25.0,2452,5.54,09:22:21,Sat. 09:32,0.18
9,Lac de St Guerin,30.0,2465,6.27,10:02:31,Sat. 10:16,0.24


In [11]:
Trails_objects[8].df_ppassage[['Point_passage','Cumul_Dist_km','Cumul_D+_m','ALL 6.9 1284 0','UTHG+TRAV 5.8 1275 0','TRAV 5.2 2137 0','UTHG 6 1283 0']].to_html('Temps_roms_UTB.html')

In [12]:
Trails_objects[8].df_ppassage[['Point_passage','Cumul_Dist_km','Cumul_D+_m','ALL 6.9 1284 0','UTHG+TRAV 5.8 1275 0','TRAV 5.2 2137 0','UTHG 6 1283 0']]

,Point_passage,Cumul_Dist_km,Cumul_D+_m,ALL 6.9 1284 0,UTHG+TRAV 5.8 1275 0,TRAV 5.2 2137 0,UTHG 6 1283 0
0,Départ,0.0,0,NaN,NaN,NaN,NaN
1,Gîte de Molliessoulaz,4.0,422,Sat. 04:54,Sat. 05:01,Sat. 04:58,Sat. 05:00
2,Les Chappes,7.0,898,Sat. 05:43,Sat. 05:54,Sat. 05:46,Sat. 05:52
3,Frêtte basse,9.0,1453,Sat. 06:26,Sat. 06:41,Sat. 06:24,Sat. 06:38
4,Col des Lacs,13.0,1771,Sat. 07:16,Sat. 07:37,Sat. 07:19,Sat. 07:33
5,Chalet du Soufflet,16.0,1777,Sat. 07:42,Sat. 08:09,Sat. 07:54,Sat. 08:03
6,Ardoisière de Cevins,18.0,1991,Sat. 08:10,Sat. 08:39,Sat. 08:23,Sat. 08:33
7,Col des Combettes,21.0,2254,Sat. 08:48,Sat. 09:22,Sat. 09:05,Sat. 09:15
8,Col des Bonnets rouges,25.0,2452,Sat. 09:32,Sat. 10:13,Sat. 09:57,Sat. 10:04
9,Lac de St Guerin,30.0,2465,Sat. 10:16,Sat. 11:05,Sat. 10:55,Sat. 10:54


In [13]:
Trails_objects[1].df_ppassage.columns.to_list()

['id',
 'Point_passage',
 'Altitude_m',
 'Cumul_Dist_km',
 'Delta_Dist_km',
 'Cumul_D+_m',
 'Delta_D+_m',
 'lon',
 'lat',
 'Heure_Passage',
 'Temps_course',
 'Classement',
 'Vitesse_km/h',
 'H_reel_h',
 'Delta_reel_h',
 'H_reel_h_init',
 'Delta_reel_h_init',
 'Trail']

In [14]:
def plot_data(i1,i2,axeX,axeY1,axeY2):
    fig,ax = plt.subplots(figsize=(16,9))
    ax.plot(Trails_objects[i1].df_ppassage[axeX],Trails_objects[i1].df_ppassage[axeY1],c='blue')
    ax.plot(Trails_objects[i2].df_ppassage[axeX],Trails_objects[i2].df_ppassage[axeY2],c='red')
    ax.set_xlabel(axeX)
    ax.set_ylabel(f'{axeY1} {axeY2}')
    plt.legend([f'{Trails_objects[i1].nom_id} {axeY1}',f'{Trails_objects[i2].nom_id} {axeY2}'])
    plt.grid()

In [15]:
Trail_selection1 = widgets.Dropdown(
    options=[(Trails_objects[i].nom_id,i) for i in range(len(Trails_objects))],
    value=6,
    description='Trail1:',
)
Trail_selection2 = widgets.Dropdown(
    options=[(Trails_objects[i].nom_id,i) for i in range(len(Trails_objects))],
    value=6,
    description='Trail2:',
)
axeX = widgets.Dropdown(
    options=Trails_objects[1].df_ppassage.columns.to_list(),
    value='Cumul_Dist_km',
    description='Axe X:',
)
axeY1 = widgets.Dropdown(
    options=Trails_objects[1].df_ppassage.columns.to_list(),
    value='H_reel_h',
    description='Axe Y1:',
)
axeY2 = widgets.Dropdown(
    options=Trails_objects[1].df_ppassage.columns.to_list(),
    value='Pred_Cum_T_Rom',
    description='Axe Y2:',
)
ui1 = widgets.HBox([Trail_selection1,axeY1,axeX])
ui2 = widgets.HBox([Trail_selection2,axeY2])
res= widgets.interactive_output(plot_data,{'i1':Trail_selection1,'i2':Trail_selection2,'axeX':axeX,'axeY1':axeY1,'axeY2':axeY2})

display(ui1,ui2,res)

TraitError: Invalid selection: value not found